In [6]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import AnyMessage, HumanMessage, AIMessage, SystemMessage
from langchain_openai import ChatOpenAI

In [7]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [37]:
req_query = "상담내용 3줄로 요약해줘"

In [38]:
llm = ChatOpenAI(model="gpt-4o", temperature=0.7, streaming=True)

In [39]:
session_id = 1234
# FIXME: curr. using history from client side

trace_id = 1234  # 내담자 기록 추적 ID


chat_history = [
    {
        "role": "assistant",
        "action": "start",
        "content": "안녕하세요, 화성시 마음 건강이입니다. 어떤 힘든 일 때문에 저를 찾아오셨나요?",
        "created_at": "2024-12-26T17:21:11.627",
    },
    {
        "role": "user",
        "action": "counsel",
        "content": "하루의 피로도가 높고... 요즘 위험한 생각도 같이 들고 있어",
        "created_at": "2024-12-26T17:21:13.123",
    },
    {
        "role": "ai",
        "action": "counsel",
        "content": "정말 힘든 하루였겠네요. 언제부터 이런 감정을 느꼈나요?",
        "created_at": "2024-12-26T17:21:14.241",
    },
    {
        "role": "user",
        "action": "counsel",
        "content": "아마 2주는 넘은 듯 한데,, 혼자 이걸 해결하려고 해도 자꾸만 무너지는 기분이 들어",
        "created_at": "2024-12-26T17:21:16.432",
    },
    {
        "role": "ai",
        "action": "counsel",
        "content": "혼자 해결하려고 애쓰셨군요. 많이 힘드셨겠어요. 마음이 답답할 때는 주로 누구와 대화를 나누나요?",
        "created_at": "2024-12-26T17:21:18.721",
    },
    {
        "role": "user",
        "action": "counsel",
        "content": "누가 날 도울 수 있겠어. 그냥 혼자서 감내하는거지",
        "created_at": "2024-12-26T17:21:20.093",
    },
    {
        "role": "ai",
        "action": "counsel",
        "content": "그렇게 느끼는 것도 괜찮아요. 하지만 당신은 혼자가 아니에요. 제가 당신을 응원하고 있어요.",
        "created_at": "2024-12-26T17:21:23.014",
    },
    {
        "role": "user",
        "action": "counsel",
        "content": "네 응원으로도 기분이 나아지질 않네... 한강에 뛰어들면,,, 바로 죽을까?",
        "created_at": "2024-12-26T17:22:07.987",
    },
    {
        "role": "ai",
        "action": "counsel",
        "content": "얼마나 힘들었으면 그런 생각을 하셨을까요? 이야기를 들으니까 걱정이 되네요. 그래도 당신의 이야기를 더 듣고 싶어요. 언제부터 자살을 생각하셨나요?",
        "created_at": "2024-12-26T17:22:10.321",
    },
    {
        "role": "ai",
        "action": "assessment",
        "content": "아니면 자가 검진을 해보시겠어요?",
        "created_at": "2024-12-26T17:22:10.918",
    },
    {
        "role": "assistant",
        "action": "assessment",
        "content": "자살 행동 척도(Suicidal Behaviors Questionnaire-Revised:SBQ-R)는 15점이고, '자살행동 위험 높음'이라는 결과가 나왔습니다. 전문가와 상담해보는 것은 어떠신가요?",
        "created_at": "2024-12-26T17:24:02.871",
    },
    {
        "role": "user",
        "action": "counsel",
        "content": "더 할 말은 없어",
        "created_at": "2024-12-26T17:24:48.231",
    },
    {
        "role": "ai",
        "action": "notice",
        "content": "언제든지 마음이 힘들면 다시 찾아주세요. 필요할 때는 상담사와 연결해 드릴 수 있어요.",
        "created_at": "2024-12-26T17:24:51.321",
    },
    {
        "role": "user",
        "action": "counsel",
        "content": "상담사와 연결하려면 어떻게 해야 해?",
        "created_at": "2024-12-26T17:24:55.231",
    },
    {
        "role": "ai",
        "action": "appointment",
        "content": "전화 상담이 필요하시면 성함과 연락처를 남겨주세요. 근무시간에 상담사와 연계해 드리겠습니다.",
        "created_at": "2024-12-26T17:24:57.123",
    },
    {
        "role": "user",
        "action": "counsel",
        "content": "내 이름은 홍길동이고, 전화번호는 010-9876-5432이야",
        "created_at": "2024-12-26T17:25:15.029",
    },
    {
        "role": "ai",
        "action": "end",
        "content": "감사합니다, 홍길동님. 남겨주신 정보를 바탕으로 상담사와 연결해 드리겠습니다. 곧 연락드릴게요.",
        "created_at": "2024-12-26T17:25:17.726",
    },
]

config = {"configurable": {"session_id": session_id, "conversation_id": trace_id}}
system_prompt = """당신은 한국의 정신건강 상담사의 업무를 지원하는 챗봇입니다.
주어진 내담자의 상담 이력을 기반으로 상담사가 요청한 업무를 수행하세요.
가이드라인:
1. 상담사가 요청한 업무가 지원 업무 내 있는지 확인
2. 지원 가능한 업무라면 요청에 대한 응답을 생성
3. 주어진 지원 업무에서 지원할 수 없는 업무라면, '해당 상담 이력에서는 지원할 수 없는 업무입니다.'라고 응답
내담자의 상담 이력:
<내담자의 상담 이력>
{user_history}
</내담자의 상담 이력>

지원 업무:
- 요약(2-3 문장)
- 주요 감정 분석(최소 2개, 최대 3개의 감정 명사)
- 주요 키워드 추출(최소 2개, 최대 5개의 명사형 키워드)
답변은 반드시 한국어로 작성해야 합니다. 답변은 50자 내외로 작성해야 합니다."""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{question}"),
    ]
)
chain = prompt | llm

In [40]:
async for chunk in chain.astream(
    {
        "question": req_query,
        "user_history": chat_history,
    },
    config,
):
    content = getattr(chunk, "content", None)
    if content and len(content) > 0:
        print(content, end="")

내담자는 최근 2주간 피로와 위험한 생각을 느꼈고, 혼자 해결하려 했으나 어려움을 겪고 있습니다. 자살 행동 척도 결과는 위험 높음으로 나왔으며, 상담사와의 연결을 요청했습니다. 홍길동이라는 이름과 연락처를 제공하여 상담사와 연결될 예정입니다.